In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Comparing high data rate w/ no interpolation to low data rate w/ interpolation data

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# Loading targets into stk from file.
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
sats = stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [3]:
# IAgStkObjectRoot root: STK Object Model Root
facility = stk_object.root.CurrentScenario.Children.New(8, 'MyFacility') # eFacility

# IAgFacility facility: Facility Object
facility.Position.AssignGeodetic(64.858875394,-147.854115119,0) # Latitude, Longitude, Altitude

# Set altitude to height of terrain
facility.UseTerrain = True

# Set altitude to a distance above the ground
facility.HeightAboveGround = 0.240  # km

In [4]:
# IAgSTKObject satellite: STK object
antenna_facility = facility.Children.New(31, 'MyAntenna') # eAntenna

# IAgAntenna antenna: Antenna object
antenna_facility.SetModel('Dipole')
antennaModel = antenna_facility.Model
antennaModel.DesignFrequency = 2.5 # GHz
antennaModel.Length = 0.01 # m
antennaModel.LengthToWavelengthRatio = 45
antennaModel.Efficiency = 85 # Percent

# IAgAntenna antenna: Antenna object
antOrientation = antenna_facility.Orientation
antOrientation.AssignAzEl(0,-90,1) # 1 represents Rotate About Boresight
antOrientation.PositionOffset.X = 0.0 # m
antOrientation.PositionOffset.Y = 1 # m
antOrientation.PositionOffset.Z = 0.25 # m

In [5]:
# IAgSTKObject satellite: STK object
receiver = sats.Children.New(17, 'MyReceiver') # eReceiver

# IAgSTKObject satellite: STK object
transmitter = facility.Children.New(24, 'MyTransmitter') # eTransmitter

# IAgTransmitter transmitter: Transmitter object
transmitter.SetModel('Complex Transmitter Model')
txModel = transmitter.Model
txModel.Frequency = 2.5 # GHz
txModel.Power = 25 # dBW
txModel.DataRate = 10 # Mb/sec

# IAgTransmitter transmitter: Transmitter object
txModel = transmitter.Model
txModel.SetModulator('BPSK')
txModel.Modulator.AutoScaleBandwidth = True

AttributeError: 'NoneType' object has no attribute 'Children'

In [ ]:
# IAgCoverageDefinition coverage: Coverage object
coverage.ComputeAccesses()

In [4]:
LLA_State = stk_object.Get_Satellite_DP("LLA State/Fixed")

- Computing_LLA State/Fixed [==========] 10/10 [100%] in 9.9s (0.96/s)          


In [5]:
LLA_State

[         time        lat           lon         alt     alt agl  lat rate  \
 0         0.0  -0.056326 -5.268993e-14  711.000021  710.982854  0.060959   
 1        60.0   3.601185 -3.221931e-01  711.083722  711.066499  0.060957   
 2       120.0   7.258339 -6.449569e-01  711.338773  711.154290  0.060948   
 3       180.0  10.914776 -9.688897e-01  711.761018  711.532070  0.060933   
 4       240.0  14.570151 -1.294629e+00  712.343579  712.010232  0.060912   
 ...       ...        ...           ...         ...         ...       ...   
 1436  86160.0  -1.596049  1.799849e+02  711.016462  710.994891 -0.060959   
 1437  86220.0  -5.253444  1.796625e+02  711.177912  711.152597 -0.060953   
 1438  86280.0  -8.910318  1.793393e+02  711.509176  711.476065 -0.060942   
 1439  86340.0 -12.566317  1.790146e+02  712.004858  711.960424 -0.060924   
 1440  86400.0 -16.221103  1.786878e+02  712.656886  712.603051 -0.060901   
 
       lon rate  alt rate  centric lat   centric lon  terrain alt     alt 